In [8]:
from keras.layers import Input, Dense, Convolution2D, MaxPooling2D, UpSampling2D, Conv2D, Flatten, Dense
from keras.models import Model, load_model
from keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img
from keras import backend as K
import keras

import numpy as np
import matplotlib.pyplot as plt

from os import listdir
from os import system
import os
import random

import imageio

In [9]:
img_width, img_height = 31, 31

nb_epoch = 50
batch_size = 16

In [10]:
base_dir = '/scratch/image_datasets/1_for_learned_sift/ridiculously_small'

train_data_dir      = base_dir + '/patches/train'
validation_data_dir = base_dir + '/patches/validation'
# test_data_dir       = base_dir + '/patches/test'

train_descrs_dir      = base_dir + '/descriptors_angles/train'
validation_descrs_dir = base_dir + '/descriptors_angles/validation'
# test_descrs_dir       = base_dir + '/descriptors_angles/test'

In [11]:
def loading_data(dir_patches, dir_descrs):
    files_patches = listdir(dir_patches + '/class0')
    files_patches.sort()
    
    files_descrs = listdir(dir_descrs + '/class0')
    files_descrs.sort()
    
    assert len(files_patches) == len(files_descrs), "The number of patches doesn't match the number of descriptors."

    patches = []
    descrs = []

    
    for file_patch, file_descr in zip(files_patches, files_descrs):
        patch = imageio.imread(dir_patches + '/class0/' + file_patch)
#         print(patch.shape)
        if patch.shape[0] == 31:
            patches.append(patch)
            descr = np.load(dir_descrs + '/class0/' + file_descr)
            descrs.append(descr)
#         elif image.shape[0] == 19:
#             temp_count19 += 1
        
        

    patches = np.array(patches)
    patches = patches.astype(np.float64) / 255
    
    descrs = np.array(descrs)
    descrs = descrs.astype(np.float64) / 255
    print("patches", patches.shape, "  descrs", descrs.shape)
    
    return patches, descrs

In [12]:
x_train, y_train = loading_data(train_data_dir, train_descrs_dir)
x_validation, y_validation = loading_data(validation_data_dir, validation_descrs_dir)

patches (35, 31, 31)   descrs (35, 129)
patches (25, 31, 31)   descrs (25, 129)


In [13]:
x_train = x_train.reshape((x_train.shape[0], x_train.shape[2], x_train.shape[2], 1))
x_validation = x_validation.reshape((x_validation.shape[0], x_validation.shape[2], x_validation.shape[2], 1))

In [14]:
x_validation.shape

(25, 31, 31, 1)

In [26]:
def MSLE_plus_plus(y_true, y_pred):
    if not K.is_tensor(y_pred):
        y_pred = K.constant(y_pred)
    y_true = K.cast(y_true, y_pred.dtype)
    first_log = K.log(K.clip(y_pred, K.epsilon(), None) + 1.)
    second_log = K.log(K.clip(y_true, K.epsilon(), None) + 1.)
    custom_loss_log = K.log(K.clip(y_true + y_pred, K.epsilon(), None) + 1.)
    custom_loss_denominator = (y_true * y_pred + 0.005) * 256  # parameters to be further adjusted
    return K.mean(K.square(first_log - second_log) + custom_loss_log / custom_loss_denominator, axis=-1)

In [15]:
input_shape = (img_width, img_height, 1)
input_img = Input(shape=input_shape)

x = Conv2D(32, (3, 3), activation="relu", padding="valid")(input_img)
x = Conv2D(32, (3, 3), activation="relu", padding="valid")(x)
x = Conv2D(32, (3, 3), activation="relu", padding="valid")(x)
x = Conv2D(32, (3, 3), activation="relu", padding="valid")(x)
x = Conv2D(16, (3, 3), activation="relu", padding="valid")(x)
x = MaxPooling2D((4, 4), padding="valid")(x)
x = Flatten(data_format="channels_last")(x)
encoded = Dense(129, activation="linear")(x)

encoder = Model(input_img, encoded)

encoder.compile(optimizer='adadelta', loss="mse")
#next up: encoder.compile(optimizer='sgd', metrics=['categorical_accuracy'], loss='categorical_crossentropy')


encoder.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 31, 31, 1)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 29, 29, 32)        320       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 27, 27, 32)        9248      
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 25, 25, 32)        9248      
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 23, 23, 32)        9248      
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 21, 21, 16)        4624      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 5, 5, 16)          0   

In [16]:
def fixed_generator(x_train, y_train, batch_size):
    while True:
        batch_list_x = []
        batch_list_y = []
        
        for i in range(x_train.shape[0]):
            batch_list_x.append(x_train[i])
            batch_list_y.append(y_train[i])
            if len(batch_list_x) == batch_size:
                yield (np.array(batch_list_x),np.array(batch_list_y))
                batch_list_x = []
                batch_list_y = []


In [17]:
model_version = '2.0.0.0.1_encoder_lastlinear_mse_adadelta_ridiculouslysmall'

os.system('mkdir ' + base_dir + '/weights' + model_version)
checkpointer = keras.callbacks.ModelCheckpoint(base_dir + '/weights' + model_version + '/weights.{epoch:02d}-{val_loss:.2f}.hdf5', monitor='val_loss', verbose=0, save_best_only=False, save_weights_only=False, mode='auto', period=1)

encoder.fit_generator(fixed_generator(x_train, y_train, batch_size),
                steps_per_epoch=x_train.shape[0],
                epochs=400,
                validation_data=fixed_generator(x_validation, y_validation, batch_size),
                validation_steps=x_validation.shape[0],
                callbacks=[checkpointer]
                )
encoder.save(base_dir + '/ae' + model_version + '.h5')

# import keras.losses
# keras.losses.MSLE_plus_plus = MSLE_plus_plus
# encoder = load_model(base_dir + '/ae' + model_version + '.h5')

Epoch 1/400
35/35 [==============================] - 1s 20ms/step - loss: 0.0277 - val_loss: 0.0205
Epoch 2/400
35/35 [==============================] - 1s 15ms/step - loss: 0.0178 - val_loss: 0.0170
Epoch 3/400
35/35 [==============================] - 1s 15ms/step - loss: 0.0167 - val_loss: 0.0169
Epoch 4/400
35/35 [==============================] - 1s 15ms/step - loss: 0.0166 - val_loss: 0.0168
Epoch 5/400
35/35 [==============================] - 1s 15ms/step - loss: 0.0166 - val_loss: 0.0168
Epoch 6/400
35/35 [==============================] - 1s 15ms/step - loss: 0.0166 - val_loss: 0.0167
Epoch 7/400
35/35 [==============================] - 1s 15ms/step - loss: 0.0165 - val_loss: 0.0168
Epoch 8/400
35/35 [==============================] - 1s 15ms/step - loss: 0.0165 - val_loss: 0.0167
Epoch 9/400
35/35 [==============================] - 1s 15ms/step - loss: 0.0164 - val_loss: 0.0168
Epoch 10/400
35/35 [==============================] - 1s 15ms/step - loss: 0.0164 - val_loss: 0.0167

35/35 [==============================] - 1s 15ms/step - loss: 0.0013 - val_loss: 0.0284
Epoch 83/400
35/35 [==============================] - 1s 15ms/step - loss: 0.0012 - val_loss: 0.0288
Epoch 84/400
35/35 [==============================] - 1s 15ms/step - loss: 0.0012 - val_loss: 0.0293
Epoch 85/400
35/35 [==============================] - 1s 15ms/step - loss: 0.0011 - val_loss: 0.0295
Epoch 86/400
35/35 [==============================] - 1s 15ms/step - loss: 0.0011 - val_loss: 0.0295
Epoch 87/400
35/35 [==============================] - 1s 15ms/step - loss: 0.0010 - val_loss: 0.0295
Epoch 88/400
35/35 [==============================] - 1s 15ms/step - loss: 9.8824e-04 - val_loss: 0.0291
Epoch 89/400
35/35 [==============================] - 1s 15ms/step - loss: 9.2952e-04 - val_loss: 0.0296
Epoch 90/400
35/35 [==============================] - 1s 15ms/step - loss: 8.3596e-04 - val_loss: 0.0294
Epoch 91/400
35/35 [==============================] - 1s 15ms/step - loss: 7.9770e-04 - val_

Epoch 160/400
35/35 [==============================] - 1s 15ms/step - loss: 5.1182e-05 - val_loss: 0.0317
Epoch 161/400
35/35 [==============================] - 1s 15ms/step - loss: 4.2446e-05 - val_loss: 0.0316
Epoch 162/400
35/35 [==============================] - 1s 15ms/step - loss: 3.0863e-05 - val_loss: 0.0317
Epoch 163/400
35/35 [==============================] - 1s 15ms/step - loss: 3.0846e-05 - val_loss: 0.0319
Epoch 164/400
35/35 [==============================] - 1s 15ms/step - loss: 3.2629e-05 - val_loss: 0.0316
Epoch 165/400
35/35 [==============================] - 1s 15ms/step - loss: 3.6476e-05 - val_loss: 0.0318
Epoch 166/400
35/35 [==============================] - 1s 14ms/step - loss: 3.5631e-05 - val_loss: 0.0318
Epoch 167/400
35/35 [==============================] - 1s 15ms/step - loss: 3.6365e-05 - val_loss: 0.0315
Epoch 168/400
35/35 [==============================] - 1s 14ms/step - loss: 3.4276e-05 - val_loss: 0.0318
Epoch 169/400
35/35 [=========================

35/35 [==============================] - 1s 15ms/step - loss: 5.2156e-06 - val_loss: 0.0317
Epoch 238/400
35/35 [==============================] - 1s 14ms/step - loss: 5.6678e-06 - val_loss: 0.0318
Epoch 239/400
35/35 [==============================] - 1s 15ms/step - loss: 5.9988e-06 - val_loss: 0.0319
Epoch 240/400
35/35 [==============================] - 1s 15ms/step - loss: 4.8391e-06 - val_loss: 0.0317
Epoch 241/400
35/35 [==============================] - 1s 15ms/step - loss: 3.7218e-06 - val_loss: 0.0318
Epoch 242/400
35/35 [==============================] - 1s 14ms/step - loss: 2.6958e-06 - val_loss: 0.0318
Epoch 243/400
35/35 [==============================] - 1s 15ms/step - loss: 2.0408e-06 - val_loss: 0.0318
Epoch 244/400
35/35 [==============================] - 1s 14ms/step - loss: 2.5294e-06 - val_loss: 0.0318
Epoch 245/400
35/35 [==============================] - 1s 15ms/step - loss: 2.4349e-06 - val_loss: 0.0318
Epoch 246/400
35/35 [==============================] - 1s 15

35/35 [==============================] - 1s 15ms/step - loss: 5.6073e-07 - val_loss: 0.0318
Epoch 315/400
35/35 [==============================] - 1s 15ms/step - loss: 4.6263e-07 - val_loss: 0.0318
Epoch 316/400
35/35 [==============================] - 1s 14ms/step - loss: 3.3963e-07 - val_loss: 0.0318
Epoch 317/400
35/35 [==============================] - 1s 15ms/step - loss: 3.3009e-07 - val_loss: 0.0318
Epoch 318/400
35/35 [==============================] - 1s 15ms/step - loss: 3.3788e-07 - val_loss: 0.0318
Epoch 319/400
35/35 [==============================] - 1s 15ms/step - loss: 3.7873e-07 - val_loss: 0.0318
Epoch 320/400
35/35 [==============================] - 1s 15ms/step - loss: 3.8607e-07 - val_loss: 0.0318
Epoch 321/400
35/35 [==============================] - 1s 15ms/step - loss: 3.9591e-07 - val_loss: 0.0318
Epoch 322/400
35/35 [==============================] - 1s 15ms/step - loss: 3.3841e-07 - val_loss: 0.0318
Epoch 323/400
35/35 [==============================] - 1s 15

35/35 [==============================] - 1s 15ms/step - loss: 8.2745e-08 - val_loss: 0.0318
Epoch 392/400
35/35 [==============================] - 1s 15ms/step - loss: 8.3653e-08 - val_loss: 0.0318
Epoch 393/400
35/35 [==============================] - 1s 15ms/step - loss: 7.6158e-08 - val_loss: 0.0318
Epoch 394/400
35/35 [==============================] - 1s 14ms/step - loss: 6.2051e-08 - val_loss: 0.0318
Epoch 395/400
35/35 [==============================] - 1s 15ms/step - loss: 4.7646e-08 - val_loss: 0.0318
Epoch 396/400
35/35 [==============================] - 1s 14ms/step - loss: 4.4220e-08 - val_loss: 0.0318
Epoch 397/400
35/35 [==============================] - 1s 15ms/step - loss: 4.1003e-08 - val_loss: 0.0318
Epoch 398/400
35/35 [==============================] - 1s 15ms/step - loss: 4.0959e-08 - val_loss: 0.0318
Epoch 399/400
35/35 [==============================] - 1s 15ms/step - loss: 4.7996e-08 - val_loss: 0.0318
Epoch 400/400
35/35 [==============================] - 1s 15

In [21]:
np.set_printoptions(suppress=True)

In [22]:
np.around(encoder.predict(imageio.imread(train_data_dir + "/class0/patch000003.bmp").reshape(1,31,31,1))*255)

array([[  6226.,  -4327.,   -212.,  -5056.,  -7646.,  -6081.,   8609.,
          7355.,   5532.,   6354.,   5112.,   1746.,    469.,   4434.,
          5450.,   1333.,   5431.,  10231.,   6492.,     11.,   3948.,
          3639.,  -1945.,  -1682.,   -183.,  -2503.,   -992.,   -431.,
          4361.,   8124.,   2575.,  -1841.,   3022.,    -79.,   1061.,
         -3145.,   2384.,   -716.,   8723.,   2711.,   7590.,  -2574.,
          2260.,   -314.,   -263.,    203.,    861.,  -2148.,   4120.,
          1577.,   1211.,  -3280.,   2762.,   7455.,   6437.,   1561.,
           170.,  -4488.,   -589.,    465.,   2837.,   2627.,   3180.,
           319.,   1134.,   -845.,  -1492.,  -2196.,  -1676.,  12213.,
          -201.,   5783.,   7160.,   1915.,    378.,   4552.,   4211.,
           557.,   4883.,   4785.,   4315.,  -4458., -12434.,  -8737.,
          5939.,   8673.,   -182.,   -775.,   3257.,  -3857.,  -2258.,
         -3089.,  -1088.,  -2382.,  -1468.,   2597.,  -1078.,  -1202.,
      

In [15]:
np.around(encoder.predict(imageio.imread(train_data_dir + "/class0/patch000003.bmp").reshape(1,31,31,1))*255)

array([[  0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0., 255.,   0.,
          0.,   0.,   0.,   0.,   0.,   0., 255.,   0.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   0.,   0., 255.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   0., 255.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   0., 255.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.]], dtype=float32)

In [14]:
np.around(encoder.predict(imageio.imread(train_data_dir + "/class0/patch000001.bmp").reshape(1,31,31,1))*255)

array([[  0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.,   0., 255., 255.,   0.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.,   0.,   0., 255.,   0.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
        255.,   0.,   0.,   0., 255., 255.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   0.,   0., 255.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   0., 255.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0., 253.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   0., 255.,   0.,   0.,   0.,   0.,   0.,   0., 252.,   0.,
          0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.]], dtype=float32)

In [21]:
np.load(train_descrs_dir + "/class0/descr_angle_000001.npy")

array([  4,   1,   0,   0, 137,  42,   1,   3,  55,   7,   0,   0,  71,
        32,   1,   3, 137,  39,   0,   0,   3,   2,   0,   5,  40,  10,
         1,  13,  81,   1,   0,   1,  11,   2,   0,   0, 137,  64,   1,
         1,  54,   2,   0,   0, 109,  23,   1,   4, 137,  33,   4,   2,
         8,   2,   0,   8,  40,  16,  12,  83,  83,   0,   0,   1,  27,
         9,   0,   1, 137,  58,   2,   0,  23,   0,   0,   0,  54,  62,
        17,  18, 137,  36,  12,   8,   5,  10,   9,  34,  10,  16,  22,
       137,  76,   2,   0,   0,  50,   8,   0,   1, 137,  57,   1,   1,
        26,   1,   0,   0,  31,  26,  18,  19,  19,   2,   4,   4,  10,
        10,  17,  47,   1,   1,   2,  27, 137,  25,   2,   3,  19],
      dtype=uint8)

In [13]:
encoder.predict(imageio.imread("/home/niaki/Code/ImageNet/tiny-imagenet-200/tiny_test16/class0/patch000026.png").reshape(1,16,16,3))

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.,
        1., 0., 0., 0., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.,
        1., 0., 1., 0., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]],
      dtype=float32)

In [45]:
temp_y_prime = encoder.predict(imageio.imread("/home/niaki/Code/ImageNet/tiny-imagenet-200/tiny_train16/class0/patch000012.png").reshape(1,16,16,3))

In [46]:
temp_y = np.load("/home/niaki/Code/ImageNet/tiny-imagenet-200/tiny_sifts/tiny_train16/class0/patch000012.npy")

In [47]:
print(np.corrcoef(temp_y, temp_y_prime[0])[1,0], "\n")
for i in range(temp_y_prime.shape[1]):
    print('{:>12f}{:>12f}'.format(temp_y[i], temp_y_prime[0, i]))      
#     print(temp_y_prime[0, i], temp_y[i])

0.6393183138370047 

    0.000000    0.000000
    0.000000    0.000000
    0.000000    0.000000
    0.000000    0.000000
    0.000000    0.000000
    0.000000    0.000000
    0.000000    0.000000
    0.000000    0.000000
    0.000000    0.000000
    0.000000    0.000000
    0.000000    0.000000
    0.000000    0.000000
    0.000000    0.000000
    0.000000    0.000000
    0.000000    0.000000
    0.000000    0.000000
    0.000000    0.000000
    0.000000    0.000000
    0.000000    0.000000
    0.000000    0.000000
    0.000000    0.000000
    0.000000    0.000000
    0.000000    0.000000
    0.000000    0.000000
    0.000000    0.000000
    0.000000    0.000000
    0.000000    0.000000
    0.000000    0.000000
    0.000000    0.000000
    0.000000    0.000000
    0.000000    0.000000
    0.000000    0.000000
    0.000000    0.000000
    0.000000    0.000000
    0.000000    0.000000
    0.000000    0.000000
    0.000000    0.000000
    0.000000    0.000000
    0.000000    0.000000
    

In [21]:
np.round(temp, decimals=9)

array([[0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00,
        0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00,
        0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00,
        0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00,
        0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00,
        0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00,
        0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 1.000e+00, 0.000e+00,
        0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00,
        0.000e+00, 0.000e+00, 3.427e-06, 0.000e+00, 0.000e+00, 0.000e+00,
        0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00,
        0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00,
        0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00,
        0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00,
        0.000e+00, 0.000e+00, 0.000e+0

In [14]:
import tensorflow as tf

print(tf.__version__)

1.14.0


In [19]:
tf.disable_v2_behavior()

W0122 20:00:43.071549 140376228677440 deprecation.py:323] From /scratch/tensorflow/lib/python3.6/site-packages/tensorflow/python/compat/v2_compat.py:61: disable_resource_variables (from tensorflow.python.ops.variable_scope) is deprecated and will be removed in a future version.
Instructions for updating:
non-resource variables are not supported in the long term


In [21]:
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.Session(config=config)